# Análisis de Datos con Agentes LangChain + Pandas

In [ ]:
#pip3 install pandas
#pip3 install tabulate

In [2]:
from langchain_ollama import ChatOllama
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_classic.agents import AgentType
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.output_parsers.pandas_dataframe import PandasDataFrameOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
import pandas as pd
import numpy as np
import os

In [3]:
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
llm = ChatOllama(model="qwen2.5:7b", temperature=0, verbose=True, disable_streaming=False)  #parece que este modeo se queda corto para estas tareas
# from langchain_google_genai import ChatGoogleGenerativeAI
# from dotenv import load_dotenv
# load_dotenv()
# llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0) 

In [5]:
#Se configura el parser: Su trabajo es decirle al modelo de lenguaje exactamente cómo debe responder para que su 
# respuesta pueda convertirse en una tabla de Pandas sin errores.
df_parser = PandasDataFrameOutputParser(dataframe=df)

In [6]:
print(df_parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

La salida debe estar formateada como una cadena como la operación, seguida de un punto y coma, seguida del nombre de la columna o fila que se quiera consultar, seguido de parámetros opcionales en array.

1. Los nombres de las columnas son limitados a los posibles columnas que se mencionan debajo.
2. Los arreglos deben ser una lista separada por comas de números formateados como [1,3,5], o estar entre corchetes de números formateados como [0..4].
3. Recuerda que los arreglos son opcionales y no siempre son necesarios.
4. Si la columna no está en los posibles nombres de columnas o la operación no es una operación válida para un DataFrame Pandas, devuelva porque es invalida como una cadena que empiece con "Invalid column" o "Invalid operation".

Como ejemplo, para las formatos:

1. La cadena `column:num_legs` es un caso bien formado que obtiene la columna `num_legs`, donde `num_legs` es una posible columna.
2. La cadena `row:1` es un caso bien formado que obtiene la fila 1.
3. La cadena `column:num_legs[1,2]` es un caso bien formado que obtiene la columna `num_legs` para las filas 1 y 2, donde `num_legs` es una posible columna.
4. La cadena `row:1[num_legs]` es un caso bien formado que obtiene la fila 1, pero solo para la columna `num_legs`, donde `num_legs` es una posible columna.
5. La cadena `mean:num_legs[1..3]` es un caso bien formado que toma el promedio de `num_legs` de las filas 1 a 3, donde `num_legs` es una posible columna y `mean` es una operación válida para un DataFrame Pandas.
6. La cadena `do_something:num_legs` es un caso malformado, ya que `do_something` no es una operación válida para un DataFrame Pandas.
7. La cadena `mean:invalid_col` es un caso malformado, ya que `invalid_col` no es una posible columna.

Aquí están los posibles nombres de columnas:

```
- PassengerId
- Survived
- Pclass
- Name
- Sex
- Age
- SibSp
- Parch
- Ticket
- Fare
- Cabin
- Embarked
```

**PREPARACIÓN DEL AGENTE**

In [55]:
prompt = PromptTemplate(template="Responde al usuario su consulta. \n{format_instructions}\n{query}\n",
                        input_variables=["query"],
                        partial_variables={"format_instructions": df_parser.get_format_instructions()})

chain = prompt | llm | df_parser

In [41]:
chain.invoke({"query":"dame la primera fila"})

{'0': PassengerId                          1
 Survived                             0
 Pclass                               3
 Name           Braund, Mr. Owen Harris
 Sex                               male
 Age                               22.0
 SibSp                                1
 Parch                                0
 Ticket                       A/5 21171
 Fare                              7.25
 Cabin                              NaN
 Embarked                             S
 Name: 0, dtype: object}

In [42]:
chain.invoke({"query":"cuál es la media de la columna Age?"})

{'mean': np.float64(29.69911764705882)}

In [43]:
df['Age'].mean()

np.float64(29.69911764705882)

In [48]:
chain.invoke({"query":"Qué tamaño tiene el df?"})

{'count': np.int64(891)}

In [50]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     889 non-null    str    
dtypes: float64(2), int64(5), str(5)
memory usage: 83.7 KB


In [62]:
# NISIQUIERA EL MODELO GEMINI2.5-FLASH es capaz de hacer algunas consultas
chain.invoke({"query":"Entregame la media de edad de los hombres "})

OutputParserException: Invalid array format in 'Age[Sex=male]'.                     Please check the format instructions.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 

In [57]:
df[df['Sex'] == 'male']['Age'].mean()

np.float64(30.72664459161148)

**PARA SOLUCIONAR LO ANTERIOR SE HARÁ EL AGENTE!!!** USANDO OLLAMA INCLUSO!!!

In [70]:
pandas_df_agent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True,
                                                handle_parsing_errors=True,
                                                agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                                return_intermediate_steps=True,
                                                allow_dangerous_code=True,
                                                prefix="Eres un asistente de análisis de datos. Siempre responde en español, sin excepción.")

c:\Users\lftob\miniconda3\envs\langchain\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [73]:
pandas_result = pandas_df_agent.invoke("qué tamaño tiene el df?")
final_answer = pandas_result["output"]
print(final_answer)



> Entering new AgentExecutor chain...
Thought: Para determinar el tamaño del DataFrame, necesito contar el número de filas en la tabla proporcionada.

Action: python_repl_ast
Action Input: len(df)891Final Answer: El DataFrame `df` tiene un tamaño de 891 filas.

> Finished chain.
El DataFrame `df` tiene un tamaño de 891 filas.


In [67]:
df.shape

(891, 12)

In [72]:
pandas_result = pandas_df_agent.invoke("cuál es la media de edad de los hombres?")
final_answer = pandas_result["output"]
print(final_answer)



> Entering new AgentExecutor chain...
Thought: Necesito filtrar el DataFrame para obtener solo a los hombres y luego calcular su media de edad.
Action: python_repl_ast
Action Input: df_men = df[df['Sex'] == 'male']; men_age_mean = df_men['Age'].mean(); men_age_mean30.72664459161148Final Answer: La media de edad de los hombres en el DataFrame es aproximadamente 30.73 años.

> Finished chain.
La media de edad de los hombres en el DataFrame es aproximadamente 30.73 años.


In [69]:
df[df['Sex'] == 'male']['Age'].mean()

np.float64(30.72664459161148)